In [13]:
import numpy as np
import pandas as pd
HASH = 'hash'
TOKEN = 'token'
min_freq = 2
min_contexts = 4
window = 3

------------

In [30]:
records = [
    ('a', 'x'), ('a', 'y'), ('a', 'z'), ('a', 'x'), ('a', 'z'),
    ('b', 'z'), ('b', 'z'), ('b', 'x'), ('b', 'p'), 
    ('c', 't'), ('c', 'z')
]
df = pd.DataFrame.from_records(records, columns=[HASH, TOKEN])
df

,hash,token
0,a,x
1,a,y
2,a,z
3,a,x
4,a,z
5,b,z
6,b,z
7,b,x
8,b,p
9,c,t


In [15]:
%%timeit

def chunk(group):
    length = len(group)
    if length <= window:
        group['chunk'] = 1
    else:
        index = np.arange(length)
        index = index % window == 0
        index = index.cumsum()
        group['chunk'] = index
    return group

df.groupby(HASH, sort=False, as_index=False).apply(chunk)#.reset_index(drop=True)

3.86 ms ± 163 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [39]:
df['hash_shift'] = df.hash.shift()
df['new_doc'] = df.hash != df.hash_shift
df['doc_id'] = df.new_doc.cumsum()
df['range'] = np.arange(len(df))
df

count = 0
for e in df[['hash', 'hash_shift']].iterrows():
    print(e)

(0, hash            a
hash_shift    NaN
Name: 0, dtype: object)
(1, hash          a
hash_shift    a
Name: 1, dtype: object)
(2, hash          a
hash_shift    a
Name: 2, dtype: object)
(3, hash          a
hash_shift    a
Name: 3, dtype: object)
(4, hash          a
hash_shift    a
Name: 4, dtype: object)
(5, hash          b
hash_shift    a
Name: 5, dtype: object)
(6, hash          b
hash_shift    b
Name: 6, dtype: object)
(7, hash          b
hash_shift    b
Name: 7, dtype: object)
(8, hash          b
hash_shift    b
Name: 8, dtype: object)
(9, hash          c
hash_shift    b
Name: 9, dtype: object)
(10, hash          c
hash_shift    c
Name: 10, dtype: object)


In [4]:
freq_global = df[TOKEN].value_counts()
freq_global

z    5
x    3
t    1
p    1
y    1
Name: token, dtype: int64

In [5]:
freq_global = freq_global[freq_global >= min_freq]
freq_global

z    5
x    3
Name: token, dtype: int64

In [6]:
df = df[df[TOKEN].isin(freq_global.index)]
df

,hash,token,chunk
0,a,x,1
2,a,z,1
3,a,x,2
4,a,z,2
5,b,z,1
6,b,z,1
7,b,x,1
10,c,z,1


In [7]:
freq_context = df.groupby([HASH, 'chunk'])[TOKEN].unique().explode().value_counts()
freq_context

z    4
x    3
Name: token, dtype: int64

In [8]:
freq_context = freq_context[freq_context >= min_contexts]
freq_context

z    4
Name: token, dtype: int64

In [9]:
df[df[TOKEN].isin(freq_context.index)]

,hash,token,chunk
2,a,z,1
4,a,z,2
5,b,z,1
6,b,z,1
10,c,z,1
